In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
class Recommender():
  def __init__(self):
      with open(os.path.join('/content/drive/MyDrive/', 'Project', 'cluster_rules.pkl'), 'rb') as file:
        self.cluster_rules = pickle.load(file)

      with open(os.path.join('/content/drive/MyDrive/', 'Project', 'rules.pkl'), 'rb') as file:
        self.universal_rules = pickle.load(file)

  def recommend_helper(self, rules, items, n_recommendations):
      # Filter rules with the input items as antecedents
      filtered_rules = rules[rules['antecedents'].apply(lambda x: x.issubset(items))]

      # Sort rules by confidence, lift, or other metric
      sorted_rules = filtered_rules.sort_values(by=['confidence', 'lift'], ascending=False)

      # Get the consequents from the rules as recommendations
      recommendations = sorted_rules['consequents'].apply(lambda x: list(x)).tolist()

      # Flatten the list and remove duplicates
      recommendations = list(set([item for sublist in recommendations for item in sublist]))

      # Remove input items from recommendations
      recommendations = [r for r in recommendations if r not in items]

      return recommendations[:n_recommendations]


  def recommend(self, items, cluster, n_recommendations=5):
      """
      Recommend new items based on a set of input items.

      Parameters:
      - items: a set of items for which to find recommendations
      - rules: precomputed association rules
      - n_recommendations: the number of recommendations to return

      Returns: a sorted list of recommended items
      """
      if cluster is None:
        return set(self.recommend_helper(self.universal_rules, items, n_recommendations))
      else:
        return set(self.recommend_helper(self.cluster_rules[cluster], items, n_recommendations) + self.recommend_helper(self.universal_rules, items, n_recommendations))


In [ ]:
from flask import Flask, request, jsonify
import os
import pickle

app = Flask(__name__)


recommender = Recommender()

# Endpoint to get recommendations
@app.route('/recommend', methods=['POST'])
def get_recommendations():
    data = request.json
    items = set(data.get('items', []))
    cluster = data.get('cluster')
    n_recommendations = data.get('n_recommendations', 5)
    recommended_items = recommender.recommend(items, cluster, n_recommendations)
    return jsonify({'recommended_items': list(recommended_items)})

# Endpoint to submit ratings for recommendations
@app.route('/rate', methods=['POST'])
def rate_recommendations():
    data = request.json
    user_id = data.get('user_id')  # Unique identifier for the user
    ratings = data.get('ratings')  # A dictionary of item: rating pairs

    # Store the ratings in a text file
    with open('/content/drive/MyDrive/Project/ratings.txt', 'a') as file:
        for item, rating in ratings.items():
            file.write(f"{user_id},{item},{rating}\n")

    return jsonify({'message': 'Ratings submitted successfully!'})

if __name__ == '__main__':
    app.run(debug=True)